In [1]:
import os
from pyspark.sql.types import *
import pyspark.sql.functions as F
import pandas as pd

# Load dataset

In [2]:
dfs = []
for db in [path for path in os.listdir('.') if 'NASA' in path]:
    dfs.append(spark.read.csv(db,sep=" "))

In [3]:
df = dfs[0].union(dfs[1])

In [4]:
df.limit(5).toPandas()

,_c0,_c1,_c2,_c3,_c4,_c5,_c6,_c7
0,in24.inetnebr.com,-,-,[01/Aug/1995:00:00:01,-0400],GET /shuttle/missions/sts-68/news/sts-68-mcc-0...,200,1839
1,uplherc.upl.com,-,-,[01/Aug/1995:00:00:07,-0400],GET / HTTP/1.0,304,0
2,uplherc.upl.com,-,-,[01/Aug/1995:00:00:08,-0400],GET /images/ksclogo-medium.gif HTTP/1.0,304,0
3,uplherc.upl.com,-,-,[01/Aug/1995:00:00:08,-0400],GET /images/MOSAIC-logosmall.gif HTTP/1.0,304,0
4,uplherc.upl.com,-,-,[01/Aug/1995:00:00:08,-0400],GET /images/USA-logosmall.gif HTTP/1.0,304,0


# Preprocessing

In [5]:
df = df.drop('_c1').drop('_c2').drop('_c4')

In [6]:
df = df.withColumnRenamed('_c0','host').withColumnRenamed('_c3','timestamp').withColumnRenamed('_c4','timezone').\
    withColumnRenamed('_c5','url').withColumnRenamed('_c6','http').withColumnRenamed('_c7','bytes')

In [7]:
df = df.withColumn("timestamp", F.regexp_replace(F.col("timestamp"), "\\[", "")).\
        withColumn("timestamp", F.substring("timestamp",1,11)).\
        withColumn("url", F.regexp_replace(F.col("url"), "GET ", "")).\
        withColumn("url", F.regexp_replace(F.col("url"), "HTTP/1.0", ""))

In [8]:
df.limit(10).toPandas()

,host,timestamp,url,http,bytes
0,in24.inetnebr.com,01/Aug/1995,/shuttle/missions/sts-68/news/sts-68-mcc-05.txt,200,1839
1,uplherc.upl.com,01/Aug/1995,/,304,0
2,uplherc.upl.com,01/Aug/1995,/images/ksclogo-medium.gif,304,0
3,uplherc.upl.com,01/Aug/1995,/images/MOSAIC-logosmall.gif,304,0
4,uplherc.upl.com,01/Aug/1995,/images/USA-logosmall.gif,304,0
5,ix-esc-ca2-07.ix.netcom.com,01/Aug/1995,/images/launch-logo.gif,200,1713
6,uplherc.upl.com,01/Aug/1995,/images/WORLD-logosmall.gif,304,0
7,slppp6.intermind.net,01/Aug/1995,/history/skylab/skylab.html,200,1687
8,piweba4y.prodigy.com,01/Aug/1995,/images/launchmedium.gif,200,11853
9,slppp6.intermind.net,01/Aug/1995,/history/skylab/skylab-small.gif,200,9202


# Número de hosts únicos.

In [9]:
df.groupBy('host').count().show()

+--------------------+-----+
|                host|count|
+--------------------+-----+
|ix-sea6-23.ix.net...|   14|
|grimnet23.idirect...|   10|
|      ird.scitex.com|   13|
|      163.205.166.15|  606|
|   chrism.tmx.com.au|    4|
| boom.marblehead.com|   27|
|        199.3.230.80|   28|
|  enigma.idirect.com| 1217|
|ip26.abq-dialin.h...|    6|
|   ppp20.coara.or.jp|   39|
|      128.159.63.129|   24|
|      132.170.244.49|   12|
|   hp165.den.mmc.com|   68|
|      128.159.143.43|  142|
|   lib-golf.tamu.edu|  208|
|       163.205.80.44|  460|
|      192.195.243.61|   42|
|   gigi.jpl.nasa.gov|    8|
|     dyna-53.bart.nl|   12|
|       164.116.78.80|   29|
+--------------------+-----+
only showing top 20 rows



In [10]:
print "Númedo de hosts únicos - {}".format(df.groupBy('host').count().count())

Númedo de hosts únicos - 137979


# O total de erros 404.

In [11]:
print "Total de erros 404 - {}".format(df.filter(F.col('http') == '404').count())

Total de erros 404 - 20871


# Os 5 URLs que mais causaram erro 404.

In [12]:
status_nf = df.filter(F.col('http')=='404').cache()

In [13]:
status_nf.groupby('url').count().orderBy(F.col('count').desc()).show(5,False)

+---------------------------------------------+-----+
|url                                          |count|
+---------------------------------------------+-----+
|/pub/winvn/readme.txt                        |2004 |
|/pub/winvn/release.txt                       |1732 |
|/shuttle/missions/STS-69/mission-STS-69.html |682  |
|/shuttle/missions/sts-68/ksc-upclose.gif     |426  |
|/history/apollo/a-001/a-001-patch-small.gif  |384  |
+---------------------------------------------+-----+
only showing top 5 rows



# Quantidade de erros 404 por dia

In [14]:
status_nf.groupBy(F.col('timestamp')).count().show()

+-----------+-----+
|  timestamp|count|
+-----------+-----+
|02/Jul/1995|  291|
|21/Aug/1995|  305|
|06/Aug/1995|  373|
|16/Jul/1995|  257|
|07/Aug/1995|  537|
|11/Aug/1995|  263|
|27/Jul/1995|  336|
|07/Jul/1995|  569|
|17/Jul/1995|  406|
|15/Jul/1995|  254|
|18/Jul/1995|  465|
|26/Jul/1995|  336|
|03/Aug/1995|  303|
|18/Aug/1995|  256|
|17/Aug/1995|  271|
|14/Aug/1995|  287|
|10/Jul/1995|  398|
|04/Jul/1995|  359|
|20/Aug/1995|  312|
|20/Jul/1995|  428|
+-----------+-----+
only showing top 20 rows



In [15]:
print 'Foram encontrados {} ocorrências de erro 404 por dia'.format(status_nf.groupBy(F.col('timestamp')).count().count())

Foram encontrados 58 ocorrências de erro 404 por dia


# O total de bytes retornados

In [16]:
print "Retornaram - {} bytes".format(df.select(F.sum('bytes')).take(1)[0][0])

Retornaram - 65524319796.0 bytes
